In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade datasets

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Constants
RANDOM_STATE = 42
K_FOLDS = 5
MODEL_NAME = "distilbert-base-uncased"
OUTPUT_DIR = "./bert_results"
METRIC_FILE = "bert_metrics.csv"

In [6]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_data(file_path='/content/drive/MyDrive/ds/data_filtered.csv'):
    """Load data from CSV file"""
    df = pd.read_csv(file_path)
    return df

def tokenize_function(example, tokenizer):
    """Tokenize text data"""
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

def compute_metrics(eval_pred):
    """Compute evaluation metrics"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

def train_and_evaluate_fold(train_df, test_df, learning_rate, num_epochs, tokenizer):
    """Train and evaluate model on a single fold"""
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    train_tokenized = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer), batched=True
    )
    test_tokenized = test_dataset.map(
        lambda x: tokenize_function(x, tokenizer), batched=True
    )

    train_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    test_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model  = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=2
    ).to(device)

    fold_output_dir = os.path.join(OUTPUT_DIR, f"lr_{learning_rate}_epochs_{num_epochs}")
    os.makedirs(fold_output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=fold_output_dir,
        report_to="none",
        eval_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir=os.path.join(fold_output_dir, 'logs'),
        logging_steps=10,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        no_cuda=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=test_tokenized,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results

def k_fold_cross_validation(df, k=K_FOLDS):
    """Perform k-fold cross-validation"""
    kf = KFold(n_splits=k, shuffle=True, random_state=RANDOM_STATE)

    learning_rates = [2e-5, 5e-5, 1e-4]
    num_epochs_list = [2, 3, 4]

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    hyperparam_results = []

    best_lr = None
    best_epochs = None
    best_avg_acc = 0

    print("Performing hyperparameter tuning on first fold...")

    for train_index, test_index in kf.split(df):
        train_df = df.iloc[train_index].reset_index(drop=True)
        test_df = df.iloc[test_index].reset_index(drop=True)

        for lr in learning_rates:
            for epochs in num_epochs_list:
                print(f"\nTraining with lr={lr}, epochs={epochs}")
                eval_results = train_and_evaluate_fold(train_df, test_df, lr, epochs, tokenizer)

                accuracy = eval_results.get('eval_accuracy', 0)
                f1 = eval_results.get('eval_f1', 0)
                precision = eval_results.get('eval_precision', 0)
                recall = eval_results.get('eval_recall', 0)

                hyperparam_results.append({
                    'learning_rate': lr,
                    'epochs': epochs,
                    'accuracy': accuracy,
                    'f1': f1,
                    'precision': precision,
                    'recall': recall
                })

                if accuracy > best_avg_acc:
                    best_avg_acc = accuracy
                    best_lr = lr
                    best_epochs = epochs

        break

    plot_hyperparameter_tuning(hyperparam_results)

    print(f"\nRunning {k}-fold cross-validation with best hyperparameters: lr={best_lr}, epochs={best_epochs}")

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    fold_results = []

    for fold, (train_index, test_index) in enumerate(kf.split(df)):
        print(f"\nTraining fold {fold + 1}/{k}")

        train_df = df.iloc[train_index].reset_index(drop=True)
        test_df = df.iloc[test_index].reset_index(drop=True)

        eval_results = train_and_evaluate_fold(train_df, test_df, best_lr, best_epochs, tokenizer)

        accuracy = eval_results.get('eval_accuracy', 0)
        f1 = eval_results.get('eval_f1', 0)
        precision = eval_results.get('eval_precision', 0)
        recall = eval_results.get('eval_recall', 0)

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        fold_results.append({
            'fold': fold + 1,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

        print(f"Fold {fold + 1} results: accuracy={accuracy:.3f}, f1={f1:.3f}, precision={precision:.3f}, recall={recall:.3f}")

    metrics = {
        'Accuracy': (np.mean(accuracies), np.std(accuracies)),
        'Precision': (np.mean(precisions), np.std(precisions)),
        'Recall': (np.mean(recalls), np.std(recalls)),
        'F1 Score': (np.mean(f1_scores), np.std(f1_scores))
    }

    print("\nFinal BERT metrics:")
    for metric, (mean, std) in metrics.items():
        print(f"{metric}: {mean:.3f} ± {std:.3f}")

    metrics_df = pd.DataFrame(fold_results)
    metrics_df.to_csv(os.path.join(OUTPUT_DIR, METRIC_FILE), index=False)

    plot_fold_results(fold_results)

    return metrics, hyperparam_results

def plot_hyperparameter_tuning(results):
    """Plot hyperparameter tuning results"""
    if not results:
        return

    df = pd.DataFrame(results)

    fig, axs = plt.subplots(2, 2, figsize=(15, 12))
    axs = axs.flatten()

    for i, epochs in enumerate(sorted(df['epochs'].unique())):
        subset = df[df['epochs'] == epochs]
        axs[0].plot(subset['learning_rate'], subset['accuracy'], 'o-',
                   label=f'{epochs} epochs', markersize=8)

    axs[0].set_title('Accuracy vs Learning Rate')
    axs[0].set_xlabel('Learning Rate')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xscale('log')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)

    for i, epochs in enumerate(sorted(df['epochs'].unique())):
        subset = df[df['epochs'] == epochs]
        axs[1].plot(subset['learning_rate'], subset['f1'], 'o-',
                   label=f'{epochs} epochs', markersize=8)

    axs[1].set_title('F1 Score vs Learning Rate')
    axs[1].set_xlabel('Learning Rate')
    axs[1].set_ylabel('F1 Score')
    axs[1].set_xscale('log')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)


    for i, lr in enumerate(sorted(df['learning_rate'].unique())):
        subset = df[df['learning_rate'] == lr]
        axs[2].plot(subset['epochs'], subset['accuracy'], 'o-',
                   label=f'LR: {lr}', markersize=8)

    axs[2].set_title('Accuracy vs Number of Epochs')
    axs[2].set_xlabel('Number of Epochs')
    axs[2].set_ylabel('Accuracy')
    axs[2].legend()
    axs[2].grid(True, alpha=0.3)


    for i, lr in enumerate(sorted(df['learning_rate'].unique())):
        subset = df[df['learning_rate'] == lr]
        axs[3].plot(subset['epochs'], subset['f1'], 'o-',
                   label=f'LR: {lr}', markersize=8)

    axs[3].set_title('F1 Score vs Number of Epochs')
    axs[3].set_xlabel('Number of Epochs')
    axs[3].set_ylabel('F1 Score')
    axs[3].legend()
    axs[3].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'hyperparameter_tuning.png'))
    plt.close()

def plot_fold_results(fold_results):
    """Plot metrics across folds"""
    df = pd.DataFrame(fold_results)

    metrics = ['accuracy', 'precision', 'recall', 'f1']

    plt.figure(figsize=(12, 8))

    for i, metric in enumerate(metrics):
        plt.plot(df['fold'], df[metric], 'o-', label=metric.capitalize(), markersize=8)

    plt.xlabel('Fold')
    plt.ylabel('Score')
    plt.title('Performance Metrics Across Folds')
    plt.xticks(df['fold'])
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'fold_results.png'))
    plt.close()

def main():
    # Load data
    print("Loading data...")
    df = load_data()
    print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

    # Perform k-fold cross-validation
    print(f"Performing {K_FOLDS}-fold cross-validation...")
    metrics, hyperparam_results = k_fold_cross_validation(df, k=K_FOLDS)

    # Print final metrics
    print("\nFinal BERT metrics:")
    for metric, (mean, std) in metrics.items():
        print(f"{metric}: {mean:.3f} ± {std:.3f}")

    # Find best hyperparameters
    best_result = max(hyperparam_results, key=lambda x: x['accuracy'])
    print(f"\nBest hyperparameters: lr={best_result['learning_rate']}, epochs={best_result['epochs']}")
    print(f"Best validation accuracy: {best_result['accuracy']:.3f}")

    print("\nDone! Results saved to CSV and PNG files.")

if __name__ == "__main__":
    main()

Loading data...
Using device: Tesla T4
Performing 5-fold cross-validation...
Performing hyperparameter tuning on first fold...

Training with lr=2e-05, epochs=2


Map:   0%|          | 0/16248 [00:00<?, ? examples/s]

Map:   0%|          | 0/4062 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.359200,0.351869,0.857706,0.408998,0.696864,0.289436
2,0.187600,0.354024,0.856721,0.512563,0.608350,0.442836


...

Map:   0%|          | 0/16248 [00:00<?, ? examples/s]

Map:   0%|          | 0/4062 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.353800,0.359310,0.856967,0.497839,0.679245,0.392906
2,0.269900,0.378053,0.861645,0.531667,0.683084,0.435198
3,0.123800,0.532535,0.850812,0.561505,0.597843,0.529332
4,0.116600,0.679528,0.847120,0.536221,0.592409,0.489768


Fold 5 results: accuracy=0.862, f1=0.532, precision=0.683, recall=0.435

Final BERT metrics:
Accuracy: 0.868 ± 0.005
Precision: 0.685 ± 0.030
Recall: 0.484 ± 0.061
F1 Score: 0.564 ± 0.035

Final BERT metrics:
Accuracy: 0.868 ± 0.005
Precision: 0.685 ± 0.030
Recall: 0.484 ± 0.061
F1 Score: 0.564 ± 0.035

Best hyperparameters: lr=5e-05, epochs=4
Best validation accuracy: 0.870

Done! Results saved to CSV and PNG files.
